# Spotify API

Importiamo le librerie necessarie:

In [74]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
import pandas as pd

In [75]:
cid = "Insert your Client ID" 
secret = "Insert your Client Secret" 

OAUTH_TOKEN_URL= "Insert your OAUTH_TOKEN" 

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(auth = OAUTH_TOKEN_URL , client_credentials_manager = client_credentials_manager , requests_timeout=20, retries=10)

Il seguente codice crea tre liste vuote: ``artist_name``, ``link`` e ``uri``. Poi viene dichiarata una lista ``mkt`` che contiene i codici di mercato di Spotify.

Successivamente, viene effettuata una ricerca sulla piattaforma Spotify usando il metodo ``sp.search()``. La ricerca viene eseguita in un ciclo "for" che scorre dall'anno 2000 al 2022. I risultati della ricerca vengono salvati in ``track_results``.

Infine, viene eseguito un ulteriore ciclo "for" per enumerare i risultati "t" e verificare se il nome dell'artista non è già presente nella lista ``artist_name``. Se non è presente, vengono aggiunti il nome dell'artista, il link esterno e l'URI alla rispettiva lista.

In [76]:
#Creo le liste dei valori che vado a chiedere all'API
artist_name = []
link = []
uri=[]


#Seleziono tutti i mercati di Spotify
mkt = [ "AD", "AR", "AT", "AU", "BE", "BG", "BO", "BR", "CA", "CH", "CL", "CO", "CR", "CY", 
      "CZ", "DE", "DK", "DO", "EC", "EE", "ES", "FI", "FR", "GB", "GR", "GT", "HK", "HN", "HU", 
      "ID", "IE", "IS", "IT", "JP", "LI", "LT", "LU", "LV", "MC", "MT", "MX", "MY", "NI", "NL", 
      "NO", "NZ", "PA", "PE", "PH", "PL", "PT", "PY", "SE", "SG", "SK", "SV", "TH", "TR", "TW", 
      "US", "UY", "VN" ]


#Utilizzo il metodo search per cercare gli artisti
for anni in range(2000,2023):
    for i in range(0,800,50):
        track_results = sp.search(q=f'year:{anni}', type='artist', limit=50, offset=i , market = mkt)
        for i, t in enumerate(track_results['artists']['items']):
            if t['name'] not in artist_name:
                artist_name.append(t['name'])
                link.append(t['external_urls']['spotify'])
                uri.append(t['uri'])

I risultati vengono salvati in un dataframe ``df``

In [79]:
df=pd.DataFrame({"Artist":artist_name, "Url" : link ,"Uri":uri})
df

Questo codice crea sei liste vuote: ``track_name``, ``artist``, ``url``, ``uri``, ``album``, e ``album_type``. Poi viene effettuato un ciclo ``for`` per scorrere la lista di artisti ``artist_name``. In ogni ciclo, viene utilizzato il metodo ``sp.artist_top_tracks()`` per ottenere le informazioni sulle tracce popolari dell'artista.

I risultati della ricerca vengono salvati in una variabile ``results``. Successivamente, viene effettuato un ulteriore ciclo ``for`` per enumerare le tracce e salvare i dettagli di ogni traccia nella rispettiva lista. I dettagli della traccia includono il nome dell'artista, il link esterno, l'URI della traccia, il nome della traccia, il nome dell'album e il tipo di album.

In [82]:
track_name = []
artist =  []
url = []
uri = []
album = []
album_type = []

#Cerco le informazioni delle Popular Tracks di ogni artista
for i in range(len(artist_name)):
    results = sp.artist_top_tracks(df.iloc[i]['Uri'])

    for track in results['tracks']:
        artist.append(df.iloc[i]['Artist'])
        url.append(df.iloc[i]['Url'])
        uri.append(track['uri'])
        track_name.append(track['name'])
        album.append(track['album']['name'])
        album_type.append(track['album']['album_type'])


In [83]:
prova = pd.DataFrame({"Artist":artist, "Url" : url ,'Track':track_name , 'Album':album, 'Album_type':album_type , 'Uri':uri})
prova

Viene effettuata una richiesta alle caratteristiche audio di ogni canzone con il metodo ``audio_features``, e vengono raccolti i valori di danceability, energia, tonalità, volume, parlato, acusticità, strumentalità, vivacità, valenza, tempo e durata. In caso di risposta null dall'API, vengono inseriti valori ``null``.

In [85]:
danceability = []
energy = []
key = []
loudness = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
duration_ms =[]

#Ottengo le Audio Features per ogni canzone, inserisco un valore null se l'api non mi restituisce un valore
for i in prova['Uri']:
    for x in sp.audio_features(tracks=[i]):
        if x != None:
            danceability.append(x['danceability'])
            energy.append(x['energy'])
            key.append(x['key'])
            loudness.append(x['loudness'])
            speechiness.append(x['speechiness'])
            acousticness.append(x['acousticness'])
            instrumentalness.append(x['instrumentalness'])
            liveness.append(x['liveness'])
            valence.append(x['valence'])
            tempo.append(x['tempo'])
            duration_ms.append(x['duration_ms'])
        else:
            danceability.append(None)
            energy.append(None)
            key.append(None)
            loudness.append(None)
            speechiness.append(None)
            acousticness.append(None)
            instrumentalness.append(None)
            liveness.append(None)
            valence.append(None)
            tempo.append(None) 
            duration_ms.append(None)

Salviamo i risultati ottenuti

In [86]:
prova=pd.DataFrame({"Artist":artist, "Url" : url ,'Track':track_name , 'Album':album, 'Album_type':album_type , "Uri": uri ,
                    'Danceability': danceability , "Energy": energy, "Key":key , "Loudness": loudness, "Speechiness": speechiness,
                    "Acousticness": acousticness , "Instrumentalness": instrumentalness, "Liveness":liveness , "Valence":valence,
                    "Tempo":tempo, "Duration_ms": duration_ms})
prova

In [88]:
prova.to_csv('songs_final.csv')